1\. Write a function that converts number representation (bin<->dec<->hex)

In [7]:
def conversion(x, type_of_x='dec'):
    
    x_str = str(x)
    
    # x is a decimal number for sure
    if (len(x_str) < 3):
        if (type_of_x == 'bin'):
            return bin(x)
        if (type_of_x == 'hex'):
            return hex(x)
        # if type is not valid (or the same as input one) then return x
        return x
            
        
    # x is a binary number for sure    
    elif (x_str[:2] == '0b'):
        if (type_of_x == 'dec'):
            return int(x, 2)
        if (type_of_x == 'hex'):
            temp = int(x, 2)
            return hex(temp)
        # if type is not valid (or the same as input one) then return x
        return x
    
    # x is a hexadecimal number for sure 
    elif (x_str[:2] == '0x'):
        if (type_of_x == 'dec'):
            return int(x, 16)
        if (type_of_x == 'bin'):
            temp = int(x, 16)
            return bin(temp)
        # if type is not valid (or the same as input one) then return x
        return x
    
    # x is a decimal number
    else:
        if (type_of_x == 'bin'):
            return bin(x)
        if (type_of_x == 'hex'):
            return hex(x)
        # if type is not valid (or the same as input one) then return x
        return x        

    
# example
n = 0b111001
t = 'hex'

# default conversion is in decimal
print(conversion(n))
print(conversion(n, t))

57
0x39


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [33]:
def conv_to_single_prec(word):
    # set the masks
    sign = 0b10000000000000000000000000000000 # pos number 31
    exp =  0b1111111100000000000000000000000  # pos between 23 and 30
    mant = 0b11111111111111111111111          # pos between 0 and 22
    add_sign = 0b100000000000000000000000

    # discover the three exceptions
    if (((word & exp) >> 23) == 255):
        if ((word & mant) == 0):
            if (((word & sign) >> 31) == 0):
                return '+INF'
            elif (((word & sign) >> 31) == 1):
                return '-INF'
        elif ((word & mant) > 0):
            return 'NaN'
    else:
        # xfloat = (-1)^sign * mant * 2 ^ (exp-127)
        # I rewrite it in 3 terms
        first_term = (-1)**((word & sign) >> 31)
        second_term = ((word & mant) | add_sign) / (2**23)
        third_term = 2**(((word & exp) >> 23) - 127)
        return first_term*second_term*third_term

bit32_word = 0b11000001101000100110011001100110
print(conv_to_single_prec(bit32_word))

-20.299999237060547


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [10]:
underflow = save_under = 1
overflow = float(1)
exponent = 0

while(True):
    if (underflow != 0):
        save_under = underflow
        underflow /= 2
    else:
        break
        
while (overflow < overflow*2):
    overflow *= 2
    exponent += 1
    
print(save_under)
print(float(2**(exponent-1)))

5e-324
8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [3]:
var = 0
adds = 0.1
new_var = var + adds

while (var != new_var):
    var = new_var
    adds /= 2
    new_var = var + adds
    
print('The machine precision is around',adds)

The machine precision is around 1.1102230246251566e-17


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [12]:
a = 0.001
b = 1000
c = 0.001

# solution of a)
def solution(a, b, c):
    
    delta = (b**2 - 4*a*c)**(1/2)
    x1 = (-b + delta)/(2*a)
    x2 = (-b - delta)/(2*a)
    
    return (x1, x2)

y = (x1, x2)= solution(a, b, c)
print('Solution of a')
print('(x1, x2) =',y)


# solution of b)
def solution_mod(a, b, c):
    
    delta = (b**2 - 4*a*c)**(1/2)
    x1 = (2*c)/(-b - delta)
    x2 = (2*c)/(-b + delta)
    
    return (x1, x2)

y = (x1, x2) = solution_mod(a, b, c)
print('Solution of b')
print('(x1, x2) =',y)


# solution of c)
def solution_exact(ax, bx, cx):
    
    delta = (b**2 - 4*a*c)**(1/2)
    # since the first formula gives the right result for the higher root (in absolute value)
    # but not for the second and the second formula gives the right result for the smaller one
    # but not for the first, we can simply combine them
    x1 = (2*c)/(-b - delta)
    x2 = (-b - delta)/(2*a)
    
    return (x1, x2)

y = (x1, x2) = solution_exact(a, b, c)
print('Solution of c')
print('(x1, x2) =',y)

Solution of a
(x1, x2) = (-9.999894245993346e-07, -999999.999999)
Solution of b
(x1, x2) = (-1.000000000001e-06, -1000010.5755125057)
Solution of c
(x1, x2) = (-1.000000000001e-06, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [18]:
x = 1
d = 1e-2


def f_of_x(point):
    return point*(point-1)

def derivate_def(function, point, delta):
    limit = (function(point+delta) - function(point))/delta
    return limit

def f_prime(point):
    return 2*point -1


# part a)
print(derivate_def(f_of_x, x, d))
print(f_prime(x))
print('The two results differs because, applying the definition, the function',
      '\nhas to treat small numbers and the machine precision is not infinite\n')


# part b)
i = 4
while (i < 15):
    d = float(str('1e-')+str(i))
    print('result with exp equal to -'+str(i))
    print(derivate_def(f_of_x, x, d))
    i += 2

1.010000000000001
1
The two results differs because, applying the definition, the function 
has to treat small numbers and the machine precision is not infinite

result with exp equal to -4
1.0000999999998899
result with exp equal to -6
1.0000009999177333
result with exp equal to -8
1.0000000039225287
result with exp equal to -10
1.000000082840371
result with exp equal to -12
1.0000889005833413
result with exp equal to -14
0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [91]:
N = 100

def function(x):
    return (1-x**2)**(1/2)


def integral_value(steps):
    integral = 0
    for i in range(1,steps):
        yk = function(-1+(2/steps)*i)
        integral += yk*(2/steps)
    return integral



import time
import math

start = time.time()
I = integral_value(N)
end = time.time()
t = (end-start)

print('The integral equals to', I)
print('It takes about', t, 'seconds')
print('The true integral is', math.pi/2)

The integral equals to 1.5691342555492505
It takes about 0.0 seconds
The true integral is 1.5707963267948966


In [92]:
cycle = 0
while (cycle < 3):
    
    start = time.time()
    I = integral_value(N)
    end = time.time()
    t = (end-start)
    
    if (t < 1 and cycle < 1):
        N *= 10
    elif (t < 1 and cycle < 2):
        N += 1000
    elif (t < 1):
        N += 10
    else:
        if (cycle < 1):
            N = int(N/10)
        elif (cycle < 2):
            N -= 100
        cycle += 1

        
print('We need an N value of about', N, 'to reach 1 seconds')
print('It takes about', t, 'seconds')
print('And the integral equals to', I)

10000000
1075000
1074970
We need an N value of about 1074970 to reach 1 seconds
It takes about 1.0064444541931152 seconds
And the integral equals to 1.5707963253027475


In [94]:
N = 8*10**(7)

start = time.time()
I = integral_value(N)
end = time.time()
t = (end-start)

print('The integral equals to', I)
print('It takes about', t, 'seconds')
print('The true integral is', math.pi/2)

The integral equals to 1.570796326793284
It takes about 92.12122631072998 seconds
The true integral is 1.5707963267948966
